In [1]:
!pip install googletrans
import googletrans

     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 931kB 9.3MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=6d652006eec279e324efa053f0290fd443e9f168feb43346c93a5d0b2e7ae40a
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=c679114a205b6830a8f1d57357d012ce99c69db6752533e199626eedec0f4ce4
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars


In [18]:
import numpy as np
import codecs
import bisect
from difflib import SequenceMatcher
from nltk.tokenize import RegexpTokenizer
from googletrans import Translator
translator = Translator(service_urls=['translate.google.co.in'])

In [3]:
path = '/content/drive/My Drive/Sentiment Analysis/'

def load_embeddings(embedding_path, glove_len):
  weight_vectors = [np.zeros((glove_len, ))]
  word_idx = {}
  with codecs.open(embedding_path, encoding='utf-8') as f:
    for line in f:
      word, vec = line.split(u' ', 1)
      word_idx[word.lower()] = len(weight_vectors)
      weight_vectors.append(np.array(vec.split(), dtype=np.float32))
  word_idx[u'-LRB-'] = word_idx.pop(u'(')
  word_idx[u'-RRB-'] = word_idx.pop(u')')
  weight_vectors.append(np.random.uniform(
      -0.05, 0.05, weight_vectors[0].shape).astype(np.float32))
  return np.asarray(np.stack(weight_vectors)), word_idx

emb_matrix, word_idx = load_embeddings(path+'Data/glove_6B_100d.txt', 100)
max_len = 56

In [17]:
word_list = list(word_idx.keys())
print(len(word_list))
max_c = 0
word_dict = {}
normal_word_dict = {}
for word in word_list:
  normal_word_dict[word] = 1
  if word_dict.get(word[0]) is None:
    word_dict[word[0]] = {}
    word_dict[word[0]][len(word)] = []
    bisect.insort(word_dict[word[0]][len(word)], word)
  elif word_dict[word[0]].get(len(word)) is None:
    word_dict[word[0]][len(word)] = []
    bisect.insort(word_dict[word[0]][len(word)], word)
  else:
    bisect.insort(word_dict[word[0]][len(word)], word)

400000


In [70]:
def process(sentence):
  tokenizer = RegexpTokenizer(r'\w+')
  sentence_words = tokenizer.tokenize(sentence)
  # print(sentence_words)
  new_words = []
  mask = []
  for word in sentence_words:
    if normal_word_dict.get(word.lower(), 0) == 0:
      hindi = translator.translate(word, src='hi', dest='hi')
      english = translator.translate(hindi.text, src='hi', dest='en')
      new_words.append(english.text.lower().split())
      mask.append(0)
    else:
      new_words.append([word])
      mask.append(1)
  # print(new_words)
  scores = []
  temp = []
  for i in range(len(new_words)):
    if mask[i]:
      temp.append(new_words[i])
      scores.append(1)
      continue
    is_word = True
    for word in new_words[i]:
      if word not in normal_word_dict:
        is_word = False
    if is_word:
      mask[i] = 1
      scores.append(2)
      temp.append(new_words[i])
      continue
    maxx = 0
    replace = ''
    for j in range(len(sentence_words[i])-1, len(sentence_words[i])+2):
      for pmatch in word_dict.get(sentence_words[i][0].lower(), {}).get(j, []):
        score = SequenceMatcher(a=pmatch, b=sentence_words[i].lower()).ratio()
        if score > maxx:
          maxx = score
          replace = pmatch
    scores.append(maxx)
    if scores[i]>0.9:
      temp.append([replace])
    else:
      temp.append([sentence_words[i]])
  # print(temp)
  # print(scores)
  final = []
  for el in temp:
    for word in el:
      final.append(word)
  return final

In [73]:
print(process('ek baar dekh bas tu isse ye itna achha phone hai, kal jaake le lenge reccommend'))

['ek', 'baar', 'dekh', 'bas', 'tu', 'isse', 'ye', 'this', 'much', 'good', 'phone', 'hai', 'kal', 'go', 'le', 'lenge', 'recommend']
